In [13]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CosmoDC2+RedMaPPer")

sys.path.insert(0, "/global/homes/c/cinlima/gcr-catalogs")
import GCRCatalogs

sys.path.insert(0, "global/homes/c/cinlima/clevar/clevar")

#Put nerc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

# from clevar import ClCatalog, MemCatalog
# from clevar.match import MembershipMatch, get_matched_pairs
# from clevar.match import output_matched_catalog
# from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf

# import clevar

from richness_mass_calib import create_richness_mass_calib

In [14]:
from getdist import plots
from getdist.mcsamples import  MCSamples
import getdist
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))

import pygtc
import pandas as pd
from IPython.display import display, Math


GetDist Version: 1.4.7, Matplotlib version: 3.9.1


In [15]:
# #Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [16]:
# # Data
# DC2_halos_m200c = fits.open(
#     "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
# )
# dt_halos = Table(DC2_halos_m200c[1].data)
# # dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']


In [17]:
# min_richness = 5
# min_halo_mass = 1e13 #Msun

# #Halos/Clusters
# #SkySim + RedMaPPer: 'skysim5000_v1.1.1_redmapper_v0.8.5'
# RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

# #SkySim: 'skysim5000_v1.1.1_image'
# halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

# #Members
# RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

# #Match catalogs
# mt = MembershipMatch()

# mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/SS_RM_match_cross_membership')

# RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

# clusters_dt = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('m200c', 'richness', 'redshift_true'))

# dt_halos = clusters_dt

test_matchdata = pd.read_csv('/global/homes/c/cinlima/MatchingCatalogs/match_catalog.csv',  sep = ';')

rich_data = test_matchdata["richness"]
z_data = test_matchdata["redshift_x"]
mass_data = test_matchdata["halo_mass"]

test_dt = Table([mass_data, rich_data, z_data], names=('m200c', 'richness', 'redshift_true'))
dt_halos = test_dt

In [10]:
ascaso = Nc.ClusterMassAscaso()
fixed_parameters_ascaso = [6] #fixing cut parameter

ascaso.param_set_by_name("cut", 1e15) #Set cut parameter value 
mset = Ncm.MSet()

lnrich_ext = Nc.ClusterMassLnrichExt()

def catalog_fit(dt_halos, model):
    rmdata = create_richness_mass_calib(dt_halos)
    dset = Ncm.Dataset.new()
    dset.append_data(rmdata)

    lh = Ncm.Likelihood.new(dset)
    
    if model:     
        mset.set(ascaso)
        rmdata.m2lnL_val(mset)  
        mset.param_set_all_ftype(Ncm.ParamType.FREE) #All parameters free

        #All parameters free except cut parameters:
        for par in fixed_parameters_ascaso:
            mset.param_set_ftype(7000, par, Ncm.ParamType.FIXED)

        mset.prepare_fparam_map()

    else:
        fixed_parameters_lnrich_ext = [12, 13, 14, 15] #fixing cut parameters

        lnrich_ext.param_set_by_name("cut", 1e15) #Set cut parameter value 
        mset.set(lnrich_ext)
        rmdata.m2lnL_val(mset)  
        mset.param_set_all_ftype(Ncm.ParamType.FREE) #All parameters free

        #All parameters free except cut parameters:
        for par in fixed_parameters_lnrich_ext:
            mset.param_set_ftype(7000, par, Ncm.ParamType.FIXED)

        mset.prepare_fparam_map()

    fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )
    fit.log_info()
    fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)
    fit.log_info()
        
    return fit

fit = catalog_fit(dt_halos, model=False)

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.19                [FREE]
#   -    muM1[01]:  0.868588963806504   [FREE]
#   -    muZ1[02]: -0.304006137332276   [FREE]
#   -    muM2[03]:  0                   [FREE]
#   -    muZ2[04]:  0                   [FREE]
#   -    muMZ[05]:  0                   [FREE]
#   -  sigma0[06]:  0.33                [FREE]
#   - sigmaM1[07]: -0.0347435585522601  [FREE]
#   - sigmaZ1[08]:  0                   [FREE]
#   - sigmaM2[09]:  0                   [FREE]
#   - sigmaZ2[10]:  0                   [FREE]
#   - sigmaMZ[11]:  0                   [FREE]
#   -      A0[12]:  0                   [FIX

In [11]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.23825598507092    [FREE]
#   -    muM1[01]:  1.46643567378369    [FREE]
#   -    muZ1[02]:  0.274583584072032   [FREE]
#   -    muM2[03]:  0.230472504999907   [FREE]
#   -    muZ2[04]: -0.717334199490655   [FREE]
#   -    muMZ[05]:  0.073145236945642   [FREE]
#   -  sigma0[06]:  0.0345836081526396  [FREE]
#   - sigmaM1[07]: -0.6829972679598     [FREE]
#   - sigmaZ1[08]:  0.117639708277245   [FREE]
#   - sigmaM2[09]: -0.0944510374431654  [FREE]
#   - sigmaZ2[10]:  0.335903081543543   [FREE]
#   - sigmaMZ[11]:  0.162732676100081   [FREE]
#   -      A0[12]:  0                   [FIX

In [ ]:
nwalkers = 1200

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("Ext.fits")

esmcmc.start_run()
esmcmc.run(800)  
esmcmc.end_run()

esmcmc.mean_covar()

# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 1200.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.23829632869939    [FREE]
#   -    muM1[01

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



    -0.0925233882967995      0.364668073357435      0.158102971917618
# NcmFitESMCMC: Catalog row 958945: m2lnL =     12168.1019069791, recalculated to     12168.1019069791, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 958945 values:      12168.1019069791       4.23423729917992       1.46476232054186      0.277775770836281       0.23036176469306     -0.653172101254422     0.0732159699137653     0.0314380617726599     -0.670594977137552     0.0436124336484654    -0.0900194337842984      0.343464618632527       0.12556291757782
# NcmFitESMCMC: Catalog row 958946: m2lnL =     12164.2868894433, recalculated to     12164.2868894433, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 958946 values:      12164.2868894433       4.24392365440919       1.47249068053269      0.283199845116292       0.23208746557137     -0.719754125228553     0.0775066559825645     0.0298902014590714     -0.694737771836176     0.0804558794511913    -0.0967796467877187      0.202597068030878      0.1404236986

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



       1.48975576542925      0.325513958502572      0.235653379780256     -0.641458777693431     0.0950701651903012     0.0420664367452854     -0.667417391820657      0.177221406475408    -0.0896823883024782      0.247549230475886      0.184122707989003
# NcmFitESMCMC: Catalog row 959107: m2lnL =     12168.2090024922, recalculated to     12168.2090024922, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 959107 values:      12168.2090024922       4.19791454200041       1.43038369779572      0.284192199852388      0.223165197277525     -0.742119558783753     0.0802117084108534     0.0679988928219736       -0.6577646127996      0.100831287545302    -0.0898727322357994       0.19969276824088      0.159211761437628
# NcmFitESMCMC: Catalog row 959108: m2lnL =     12180.9348433544, recalculated to     12180.9348433544, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 959108 values:      12180.9348433544       4.24044329568443       1.46470601125035      0.353247303377696      0.2295316395

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



# NcmFitESMCMC: Catalog row 959611: m2lnL =     12172.7182277546, recalculated to     12172.7182277546, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 959611 values:      12172.7182277546       4.24695340378119       1.47635352880878      0.208912282988226      0.233329475759435     -0.837962989972055     0.0465093586010229     0.0474632107658049     -0.668561241724273      0.092833833021673    -0.0917759317777549      0.376569623289375      0.139849780988999
# NcmFitESMCMC: Catalog row 959612: m2lnL =     12165.1739062565, recalculated to     12165.1739062565, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 959612 values:      12165.1739062565       4.24478053016654       1.47316010188962      0.312462652757224      0.231752141050492     -0.618586061133332     0.0893186084701575     0.0184897663741355     -0.691666283432588      0.174798502972928    -0.0954248531599105      0.314788791812491      0.193832455301317
# NcmFitESMCMC: Catalog row 959613: m2lnL =     12165.9646862073

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



    0.0142586367195056     -0.703609752482246      0.100200153928693    -0.0994020547258955      0.418541398299587      0.149908205670368
# NcmFitESMCMC: Catalog row 959773: m2lnL =     12169.8628633522, recalculated to     12169.8628633522, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 959773 values:      12169.8628633522       4.22295935130791       1.45878488347651      0.207412192716666      0.229396774209414     -0.630283389571256       0.05030685102461      0.037253415570528     -0.684496990698163      0.212378038183267    -0.0956503163917756      0.451144065413792      0.202949896811689
# NcmFitESMCMC: Catalog row 959774: m2lnL =     12167.9220284945, recalculated to     12167.9220284945, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 959774 values:      12167.9220284945       4.24231386860896       1.47129994135311      0.249086594891629      0.231461977803472     -0.631380434332692     0.0656752345809655     0.0321947879578443     -0.675824204761833      0.08050544168

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
fit = catalog_fit(dt_halos, model=True)

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  3.19                [FREE]
#   -    mup1[01]:  0.868588963806504   [FREE]
#   -    mup2[02]: -0.304006137332276   [FREE]
#   - sigmap0[03]:  0.33                [FREE]
#   - sigmap1[04]: -0.0347435585522601  [FREE]
#   - sigmap2[05]:  0                   [FREE]
#   -     cut[06]:  1e+15               [FIXED]
#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
#........................................................

In [9]:
nwalkers = 1000

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("Ascaso.fits")

esmcmc.start_run()
esmcmc.run(500)  
esmcmc.end_run()

esmcmc.mean_covar()


# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 1000.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  3.03813575534708    [FREE]
#   -    mup1[01]: 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  2.4784e-05   9.7864e-06   3.7046e-06   1.9535e-05 
# NcmMSetCatalog: Current sd:     360.43       0.014008     0.0046118    0.015438     0.0060959    0.0023076    0.012168   
# NcmMSetCatalog: Current var:    1.2991e+05   0.00019623   2.1268e-05   0.00023832   3.716e-05    5.3249e-06   0.00014807 
# NcmMSetCatalog: Current tau:    1            1            1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  1            1.0135       1.0162       1.0102       1.0141       1.013        1.0123     
# NcmMSetCatalog: Maximal Shrink factor = 1.01667               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `388':
# NcmMSetCatalog: Constant break point at `82':
# NcmFitESMCMC:acceptance ratio 37.4406% (last update 44.1000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC:last ensemble variance of -2ln(L): 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



#  Elapsed time: 00 days, 00:00:00.3115940
#  iteration            [000127]
#  function evaluations [000129]
#  gradient evaluations [000000]
#  degrees of freedom   [039594]
#  m2lnL     =     20848.0287514663 (     20841.847 )
#  Fit parameters:
#     3.03834315718042       0.359079851417819      0.0346893800022423     0.642297032012666      0.105670360879847      0.107687902469182     
# NcmMSetCatalog: Current mean:   20867        3.0383       0.35906      0.035389     0.64226      0.10565      0.10754    
# NcmMSetCatalog: Current msd:    0.48093      1.9241e-05   6.3844e-06   2.1913e-05   8.5498e-06   3.2196e-06   1.7064e-05 
# NcmMSetCatalog: Current sd:     328.65       0.013149     0.0043629    0.014974     0.0058427    0.0022002    0.011661   
# NcmMSetCatalog: Current var:    1.0801e+05   0.00017289   1.9035e-05   0.00022423   3.4137e-05   4.841e-06    0.00013598 
# NcmMSetCatalog: Current tau:    1            1            1            1            1            1            

In [34]:
nwalkers = 1200
burnin = 40

ntests = 100.0

mcat = Ncm.MSetCatalog.new_from_file_ro("Ext.fits", nwalkers * burnin)

mcat.log_current_chain_stats()
mcat.calc_max_ess_time(ntests, Ncm.FitRunMsgs.SIMPLE)
mcat.calc_heidel_diag(ntests, 0.0, Ncm.FitRunMsgs.SIMPLE)

# mcat.log_full_covar()
mcat.log_current_stats()

be1, post_lnnorm_sd = mcat.get_post_lnnorm()
lnevol, glnvol = mcat.get_post_lnvol(0.6827)

Ncm.cfg_msg_sepa()
print(
    "# Bayesian evidence:                                 % 22.15g +/- % 22.15g"
    % (be1, post_lnnorm_sd)
)
print("# 1 sigma posterior volume:                          % 22.15g" % lnevol)
print("# 1 sigma posterior volume (Gaussian approximation): % 22.15g" % glnvol)

# NcmMSetCatalog: Current skfac:  1.0071       1.0085       1.0083       1.0086       1.0082       1.0086       1.0088       1.0094       1.009        1.0093       1.0088       1.0093       1.0092     
# NcmMSetCatalog: Maximal Shrink factor = 1.01064               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `760':
# NcmMSetCatalog: Constant break point at `0':
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Calculating catalog effective sample size from chain 760 => 0 using 100 blocks:
# NcmMSetCatalog: - best cutoff time:         0   
# NcmMSetCatalog: - total number of points:   760  (912000)
# NcmMSetCatalog: - number of points left:    760  (912000)
# NcmMSetCatalog: - worst parameter:          NcClusterMass:muMZ[06]
# NcmMSetCatalog: - worst parameter ess:      44.58
# NcmMSetCatalog: - worst parameter a

In [35]:
nwalkers = 1000
burnin = 90

ntests = 100.0

mcat = Ncm.MSetCatalog.new_from_file_ro("Ascaso.fits", nwalkers * burnin)

mcat.log_current_chain_stats()
mcat.calc_max_ess_time(ntests, Ncm.FitRunMsgs.SIMPLE)
mcat.calc_heidel_diag(ntests, 0.0, Ncm.FitRunMsgs.SIMPLE)

# mset.pretty_log()
# mcat.log_full_covar()
mcat.log_current_stats()

be2, post_lnnorm_sd = mcat.get_post_lnnorm()
lnevol, glnvol = mcat.get_post_lnvol(0.6827)

Ncm.cfg_msg_sepa()
print(
    "# Bayesian evidence:                                 % 22.15g +/- % 22.15g"
    % (be2, post_lnnorm_sd)
)
print("# 1 sigma posterior volume:                          % 22.15g" % lnevol)
print("# 1 sigma posterior volume (Gaussian approximation): % 22.15g" % glnvol)

# NcmMSetCatalog: Current skfac:  1.0028       1.0032       1.0032       1.003        1.003        1.003        1.0032     
# NcmMSetCatalog: Maximal Shrink factor = 1.00337               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `410':
# NcmMSetCatalog: Constant break point at `0':
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Calculating catalog effective sample size from chain 410 => 0 using 100 blocks:
# NcmMSetCatalog: - best cutoff time:         5   
# NcmMSetCatalog: - total number of points:   410  (410000)
# NcmMSetCatalog: - number of points left:    405  (405000)
# NcmMSetCatalog: - worst parameter:          NcClusterMass:sigmap0[04]
# NcmMSetCatalog: - worst parameter ess:      99.52
# NcmMSetCatalog: - worst parameter ar order: 1   
# NcmMSetCatalog: - ess's:                    115.47 108.97 1

In [37]:
np.exp(be1 - be2)

/tmp/ipykernel_1207119/2265718628.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(be1 - be2)


inf

In [ ]:
data_fit_full = pd.DataFrame(fits.open("Ascaso.fits")[1].data).iloc[:, 1:7].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
]
labels=["\mu_{0}", "\mu_{1}", "\mu_{2}", "\sigma_{0}", "\sigma_{1}", "\sigma_{2}"]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples1 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples1.removeBurn(0.3)

In [ ]:
# Triangle plot
g1 = plots.get_subplot_plotter()

g1.settings.legend_fontsize = 18
g1.settings.lab_fontsize = 25
g1.settings.axes_fontsize = 20


g1.triangle_plot(
    [samples1],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fontsize=14,
    fine_bins=1,
    colors=['#a0d995'],
    line_args=[{'lw':1.2,'ls':'-', 'color':'#a0d995'}],
    legend_labels=["Ascaso"],
)

plt.show()
for i in range(1, 7):
    display(Math(samples1.getInlineLatex(str(i),limit=1)))
plt.savefig('mcmc3.png')



In [ ]:
data_fit_full = pd.DataFrame(fits.open("Ext.fits")[1].data).iloc[:, 1:13].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    '10',
    '11',
    '12'
]
labels=["\mu_0", "\mu_{M1}", "\mu_{M2}", "\mu_{Z1}"," \mu_{Z2}", "\mu_{MZ}", "\sigma_0", "\sigma_{M1}", "\sigma_{M2}", "\sigma_{Z1}", "\sigma_{Z2}", "\sigma_{MZ}" ]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples2 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples2.removeBurn(0.3)


In [ ]:
# Triangle plot
g2 = plots.get_subplot_plotter()

g2.settings.lab_fontsize = 40
g2.settings.legend_fontsize = 40
g2.settings.axes_fontsize = 30

g2.triangle_plot(
    [samples2],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fine_bins=1,
    colors=['#b186f1'],
    fontsize=70,
    line_args=[{'lw':1.2,'ls':'-', 'color':'#b186f1'}],
    legend_labels=["Estendido", ],
)
#plt.savefig('mcmc.png')
plt.show()


In [ ]:
for i in range(1, 14):
    display(Math(samples2.getInlineLatex(str(i),limit=1)))


In [ ]:
# min_richness = 5
# min_halo_mass = 1e13 #Msun

# #Halos/Clusters
# #CosmoDC2 + RedMaPPer: 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
# RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

# #CosmoDC2: 'cosmoDC2_v1.1.4_image'
# halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

# #Members
# RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

# #Match catalogs
# mt = MembershipMatch()

# mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/cosmoDC2_RM_match_cross_membership')

# #Matched Data
# RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

# cdc2_dt = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('m200c', 'richness', 'redshift_true'))
# dt_halos = cdc2_dt


# fit = catalog_fit(dt_halos, ascaso=False)


In [ ]:
# nwalkers = 1500

# Ncm.func_eval_set_max_threads(2)
# Ncm.func_eval_log_pool_stats()

# init_sampler = Ncm.MSetTransKernGauss.new(0)
# init_sampler.set_mset(mset)
# init_sampler.set_prior_from_mset()
# init_sampler.set_cov_from_rescale(1.0)

# apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

# esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
# esmcmc.set_nthreads(2)
# esmcmc.set_data_file("mcmc2.fits")

# esmcmc.start_run()
# esmcmc.run(1000)  
# esmcmc.end_run()

# esmcmc.mean_covar()


In [ ]:
fit.log_covar()

In [ ]:
data_fit_full = pd.DataFrame(fits.open("mcmc2.fits")[1].data).iloc[:, 1:14].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    '10',
    '11',
    '12',
    '13'
]
labels=["\mu_0", "\mu_{M1}", "\mu_{M2}", "\mu_{Z1}"," \mu_{Z2}", "\mu_{MZ}", "\sigma_0", "\sigma_{M1}", "\sigma_{M2}", "\sigma_{Z1}", "\sigma_{Z2}", "\sigma_{MZ}","A_0" ]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples3 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples3.removeBurn(0.3)

In [ ]:
# Triangle plot
g2 = plots.get_subplot_plotter()

g2.settings.lab_fontsize = 40
g2.settings.legend_fontsize = 40
g2.settings.axes_fontsize = 30

g2.triangle_plot(
    [samples3],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fine_bins=1,
    colors=['#b186f1'],
    fontsize=70,
    line_args=[{'lw':1.2,'ls':'-', 'color':'#b186f1'}],
    legend_labels=["Estendido", ],
)
plt.savefig('mcmc2.png')
plt.show()


In [ ]:
for i in range(1, 14):
    display(Math(samples3.getInlineLatex(str(i),limit=1)))
